# Federated Forest

This file imports the fully preprocessed data (preprocessing IV) and applies sklearn's random forest classifiers on each of the ethnicities (representing local models). These models are stored and aggregated in the federated model (global model) such that each ethnicity is equally weighted. Consequently, the performance of the local models and the global model is evaluated using accuracy, recall, precision and AUC (ROC).

In [1]:
import math
import glob
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

## Import data

In [2]:
ethnic_group_names = ['unknown', 'white', 'other', 'asian', 'hispanic_latino', 'black_african_american', 'unable_to_obtain', 'american_indian_alaska_native']
path = './data/preprocessing_IV/'

def read_train_val_test(name, path):
    train = pd.read_csv(path + name + '_train.csv')
    # NOTE: drope datetime columns for now, because sklearn does not support it
    train = train.drop(columns=['admittime', 'edregtime', 'emar_charttime', 'lab_charttime'])
    
    validate = pd.read_csv(path + name + '_validate.csv')
    # NOTE: drope datetime columns for now, because sklearn does not support it
    validate = validate.drop(columns=['admittime', 'edregtime', 'emar_charttime', 'lab_charttime'])
    
    test = pd.read_csv(path + name + '_test.csv')
    # NOTE: drope datetime columns for now, because sklearn does not support it
    test = test.drop(columns=['admittime', 'edregtime', 'emar_charttime', 'lab_charttime'])

    return train, validate, test

### Train each local model

In [3]:
# split features and labels
def splitFeaturesLabels(df):
    X = df.copy()
    X = X.drop('has_kidney_issue', axis=1)
    
    return X, df.has_kidney_issue

In [4]:
def trainModel(name, path, all_params):
    # get data and optimal parameters
    train, val, test = read_train_val_test(name, path)
    hyperparams = all_params.loc[all_params['ethnicity']==name]
    X_train, y_train = splitFeaturesLabels(train)
    
    # train a model with the specified params
    clf = RandomForestClassifier(n_estimators=int(hyperparams['n_estimators']),
                                 max_depth=int(hyperparams['max_depth']),
                                 random_state=42)
    clf.fit(X_train, y_train)
    
    return clf

In [5]:
models = []
params = pd.read_csv('hyperparameters.csv')
for eth in ethnic_group_names:
    models.append(trainModel(eth, path, params))

### Aggregate models into federated forest

In [6]:
def aggregateForests(agg, model, n):
    # Aggregate the estimators
    size = model.n_estimators
    dupl = math.floor(n/size)
    for i in range(dupl):
        agg.estimators_ += model.estimators_
        agg.n_estimators = len(agg.estimators_)
        
    return agg

In [7]:
def getMaxForest(model_list):
    # normalize over the maximum number of trees so each model is weighted the same in the aggregated model
    sizes = [len(model.estimators_) for model in model_list]
    
    return max(sizes)

In [8]:
def federatedForest(model_list):
    # Aggregate all models and normalize
    ff = model_list[0]
    max_size = getMaxForest(model_list)
    for i, model in enumerate(model_list):
        if i > 0:
            ff = aggregateForests(ff, model_list[i], max_size)
            
    return ff

In [9]:
fed = federatedForest(models)

### Model evaluations

In [10]:
def evaluateMetrics(model, X, y):
    y_pr = model.predict(X)
    accuracy = accuracy_score(y, y_pr)
    precision = precision_score(y, y_pr)
    recall = recall_score(y, y_pr)
    auc = roc_auc_score(y, y_pr)
    
    return accuracy, precision, recall, auc

In [11]:
def printMetrics(model, ethnicity, path):
    train, val, test = read_train_val_test(ethnicity, path)
    X, y = splitFeaturesLabels(test)
    acc, prec, rec, auc = evaluateMetrics(model, X, y)
    
    print('Local model:\t\t' + ethnicity)
    print('Accuracy: \t\t\t' + str(acc))
    print('Precision: \t\t\t' + str(prec))
    print('Recall: \t\t\t' + str(rec))
    print('AUC: \t\t\t\t' + str(auc) + '\n\n')

In [12]:
def printFedMetrics(model, ethnicity, path):
    train, val, test = read_train_val_test(ethnicity, path)
    X, y = splitFeaturesLabels(test)
    acc, prec, rec, auc = evaluateMetrics(model, X, y)
    
    print('Federated model: \t' + ethnicity)
    print('Accuracy: \t\t\t' + str(acc))
    print('Precision: \t\t\t' + str(prec))
    print('Recall: \t\t\t' + str(rec))
    print('AUC: \t\t\t\t' + str(auc) + '\n\n')

In [13]:
def metricsDF(model_type, model, ethnicity, path):
    train, val, test = read_train_val_test(ethnicity, path)
    X, y = splitFeaturesLabels(test)
    acc, prec, rec, auc = evaluateMetrics(model, X, y)
    
    return [model_type, ethnicity, round(acc, 4), round(prec,4), round(rec, 4), round(auc, 4)]

In [17]:
dfs = []
columns = ['Model', 'Ethnicity', 'Accuracy', 'Precision', 'Recall', 'AUC']

for i in range(len(models)):
    # Since 'unknown' is overwritten as the aggregated model, redefine it
    if i == 0:
        unknown = trainModel('unknown', path, params)
        printMetrics(unknown, ethnic_group_names[i], path)
        dfs.append(metricsDF('Local', unknown, ethnic_group_names[i], path))
    else:
        printMetrics(models[i], ethnic_group_names[i], path)
        dfs.append(metricsDF('Local', models[i], ethnic_group_names[i], path))
        
    printFedMetrics(fed, ethnic_group_names[i], path)

    dfs.append(metricsDF('Federated', fed, ethnic_group_names[i], path))

Local model:		unknown
Accuracy: 			0.8851699279093718
Precision: 			0.6855895196506551
Recall: 			0.5097402597402597
AUC: 				0.7328383061247199


Federated model: 	unknown
Accuracy: 			0.8748712667353244
Precision: 			0.6006191950464397
Recall: 			0.6298701298701299
AUC: 				0.7754613807245386


Local model:		white
Accuracy: 			0.8144894095373328
Precision: 			0.7039790118058592
Recall: 			0.3957232395231658
AUC: 				0.6714853389340611


Federated model: 	white
Accuracy: 			0.8155543722636375
Precision: 			0.6778900112233446
Recall: 			0.445372987587563
AUC: 				0.689141474897923


Local model:		other
Accuracy: 			0.8319672131147541
Precision: 			0.5949367088607594
Recall: 			0.2842741935483871
AUC: 				0.6201992539954001


Federated model: 	other
Accuracy: 			0.8438897168405365
Precision: 			0.6222222222222222
Recall: 			0.3951612903225806
AUC: 				0.6703868608834109


Local model:		asian
Accuracy: 			0.8715334420880914
Precision: 			0.6263345195729537
Recall: 			0.45595854922279794


In [24]:
# add results to dataframe
df = pd.DataFrame(dfs, columns=columns)
df.loc['Local mean'] = df[df['Model']=='Local'].mean()
df.loc['Federated mean'] = df[df['Model']=='Federated'].mean()

# store as csv
df.to_csv('./metrics.csv', index=False)
df

C:\Users\annik\AppData\Local\Temp\ipykernel_9272\1199555007.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.loc['Local mean'] = df[df['Model']=='Local'].mean()
C:\Users\annik\AppData\Local\Temp\ipykernel_9272\1199555007.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.loc['Federated mean'] = df[df['Model']=='Federated'].mean()


,Model,Ethnicity,Accuracy,Precision,Recall,AUC
0,Local,unknown,0.885200,0.685600,0.509700,0.732800
1,Federated,unknown,0.874900,0.600600,0.629900,0.775500
2,Local,white,0.814500,0.704000,0.395700,0.671500
3,Federated,white,0.815600,0.677900,0.445400,0.689100
4,Local,other,0.832000,0.594900,0.284300,0.620200
5,Federated,other,0.843900,0.622200,0.395200,0.670400
6,Local,asian,0.871500,0.626300,0.456000,0.702600
7,Federated,asian,0.875200,0.669500,0.409300,0.685800
8,Local,hispanic_latino,0.834100,0.688500,0.301300,0.633800
9,Federated,hispanic_latino,0.839100,0.711600,0.319900,0.643900
